In [1]:
import numpy as np
import pandas as pd
import requests

from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

In [42]:
# Import Data
print('Load Description Embeddings')
desc_emb = np.load('../data/desc_emb.npz')
traindata_desc_embeddings = dict(zip(("desc_embeddings"), (desc_emb[k] for k in desc_emb)))['d']
print(traindata_desc_embeddings.shape)

print('Load Description Labels, Address Labels and Name Labels')
NFT_attributes = np.load('../data/NFT_attributes.npz')
d = dict(zip(("description_list","contract_addresses","name_list"), (NFT_attributes[k] for k in NFT_attributes)))
description_list = d['description_list']
contract_addresses = d['contract_addresses']
name_list = d['name_list']
print(description_list.shape)

train_desc_address = list(zip(contract_addresses, traindata_desc_embeddings, description_list))
#print(train_desc_address[0])

Load Description Embeddings
(11083, 768)
Load Description Labels, Address Labels and Name Labels
(11083,)


In [31]:
# Here we store the address of the person logging in with MetaMask 
OWNER = '0xe11BFCBDd43745d4Aa6f4f18E24aD24f4623af04'

URL = f'https://eth-mainnet.alchemyapi.io/v2/demo/getNFTs/?owner={OWNER}'
Owner_NFTs = requests.get(URL).json()['ownedNfts']

#print(Owner_NFTs[0]['contract']['address'])

#Format is (Contractaddress, tokenId)
owner_nfts = []

for nft in Owner_NFTs:
    nft_contract_address = nft['contract']['address'] 
    token_id = nft['id']['tokenId']
    owner_nfts.append((nft_contract_address, token_id))

In [32]:
# Get Description of All NFTs of Owner
owner_nfts_withDesc = []
only_descriptions = []

for nft in owner_nfts:
    nft_contract_address = nft[0]
    token_id = nft[1]
    URL = f"https://eth-mainnet.alchemyapi.io/v2/demo/getNFTMetadata?contractAddress={address}&tokenId={token_id}"
    NFT = requests.get(URL)
    try:
        NFTjson = NFT.json()
    except:
        continue
    if (type(NFTjson) is dict and "metadata" in NFTjson.keys() and type(NFTjson["metadata"]) is dict and "description" in NFTjson["metadata"].keys()):
        description = NFTjson["metadata"]["description"]
        only_descriptions.append(description)
        owner_nfts_withDesc.append((nft_contract_address, token_id, description))
        #print('Desc:', description

print(owner_nfts_withDesc[0])

('0x000386e3f7559d9b6a2f5c46b4ad1a9587d59dc3', '0x0000000000000000000000000000000000000000000000000000000000000003', 'A collection of 8,888 colorful undead Apefellaz looking for a new home. Each Apefellaz is jammed packed with exclusive utility, alpha and perks such as our future Metaverse development. Apefellaz is made with love by proud Deadfellaz holders. It is not affiliated with any other projects and we are continually inspired by the works of these amazing collections. All about spreading happiness, positivity, and good vibes.. come join the family!')


In [44]:
# Option 1: User then chooses a specific NFT and we provide X (e.g. X = 5) recommendations

X = 5

#Set dependent on User Input
owner_nfts_withDesc_index = 0
nft_description = owner_nfts_withDesc[0]

# Later on here we will just load out finetuned model from above
model = SentenceTransformer('bert-base-nli-mean-tokens')

#description embeddings
nft_description_embedding = model.encode(nft_description, normalize_embeddings=True, show_progress_bar=True, batch_size = 64)

print(nft_description_embedding)

#now a quadruple of distance, embedded desc, desc and address but later will also have id
recommendations = []
total_recommendations = 0

for i, j in enumerate(train_desc_address):
    train_address = j[0]
    train_description_embedding = j[1]
    train_description = j[2]

    distance = np.sum(np.square((train_description_embedding - nft_description_embedding)))

    if (i < X):
        recommendations.append((distance, train_description_embedding, train_description, train_address))
        recommendations.sort(key = lambda x: x[0])
    else:
        highest_distance = recommendations[X-1][0]
        if (distance < highest_distance):
            recommendations[X-1] = (distance, train_description_embedding, train_description, train_address)
            recommendations.sort(key = lambda x: x[0])


print("Input NFT Description: ", nft_description)
print(f"Output NFT Descriptions for top {X} matches", list(zip(*recommendations))[2])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[ 0.01741892  0.03565018  0.05280961 ... -0.01533552  0.03399569
   0.05235408]
 [ 0.00971485  0.00795302  0.05625482 ... -0.01624798 -0.01517915
   0.03131065]
 [-0.03011487  0.07835224  0.03461941 ...  0.0354766   0.0146795
  -0.00573328]]
Input NFT Description:  ('0x000386e3f7559d9b6a2f5c46b4ad1a9587d59dc3', '0x0000000000000000000000000000000000000000000000000000000000000003', 'A collection of 8,888 colorful undead Apefellaz looking for a new home. Each Apefellaz is jammed packed with exclusive utility, alpha and perks such as our future Metaverse development. Apefellaz is made with love by proud Deadfellaz holders. It is not affiliated with any other projects and we are continually inspired by the works of these amazing collections. All about spreading happiness, positivity, and good vibes.. come join the family!')
Output NFT Descriptions for top 5 matches ('**0x64931f06d3266049bf0195346973762e6996d764** &NewLine;&NewLine; [SVG](http://api.tiles.art/svg/64931f06d3266049bf019534697

In [ ]:
# Option 2: For all the NFTs the User have we find the X (e.g. X = 5) NFTs that are closest to any NFTs he already has



print('Loading Model')
model = SentenceTransformer('bert-base-nli-mean-tokens')

#description embeddings
print(f'Calculating embeddings for {len(Owner_descriptions)} descriptions')
Owner_desc_embeddings = model.encode(Owner_descriptions, normalize_embeddings=True, show_progress_bar=True, batch_size = 64)

In [ ]:
print('Calculating Recommendations')

recommendatons = []

for i, owner_emb in enumerate(Owner_desc_embeddings):
    min_index = 0
    min_distance = 10

    for j, dj in enumerate(desc_embeddings):
        distance = np.sum(np.square((owner_emb - dj)))
        if distance < min_distance and distance > 1e-6:
            min_distance = distance
            min_index = j
    recommendatons.append(min_index)
    print(f'The closest NFT for Owner_NFT_{i} is All_NFT_{min_index} with a distance of {min_distance}')
    print(f'\ndescription Owner_NFT_{i}: {Owner_descriptions[:100]}')
    print(f'\ndescription All_NFT_{min_index}: {description_list[min_index][:100]}')
    print("\n\n")